In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import os
from google.colab.patches import cv2_imshow

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras. preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten


In [ ]:
!unzip -q "/content/drive/MyDrive/DL/data.zip" -d "/content/drive/MyDrive/DL/dataset"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = ImageDataGenerator(rescale = 1/255)
validation = ImageDataGenerator(rescale = 1/255)

In [ ]:
train_dataset = train.flow_from_directory("/content/drive/MyDrive/DL/dataset/train", target_size = (256,256),batch_size = 100, class_mode ='binary')

Found 46978 images belonging to 2 classes.


In [ ]:
val_dataset = train.flow_from_directory("/content/drive/MyDrive/DL/dataset/val", target_size = (256,256),batch_size = 100, class_mode ='binary')

Found 2000 images belonging to 2 classes.


In [ ]:
train_dataset.class_indices

{'Signed': 0, 'Unsigned': 1}

In [ ]:

model = Sequential()
model.add(Conv2D(input_shape=(256,256,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [ ]:
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2048,activation="relu"))
model.add(Dense(units=1024,activation="relu"))

model.add(Dense(units=1, activation="sigmoid"))

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss = 'binary_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [ ]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 64, 64, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 256)       2

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                patience=5)

In [ ]:
model_fit = model.fit(train_dataset,
                      steps_per_epoch = 100,
                     epochs = 100,
                    verbose = 1,
                     validation_data = val_dataset,
                      callbacks = early_stopping)


Epoch 1/100
100/100 [==============================] - 109s 1s/step - loss: 0.2668 - accuracy: 0.9001 - val_loss: 0.6361 - val_accuracy: 0.7735
Epoch 2/100
 47/100 [=============>................] - ETA: 52s - loss: 0.2312 - accuracy: 0.9121

In [ ]:
model.save('/content/drive/MyDrive/DL/dataset/model.h5')